## 2011 play by play data

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import datetime, time
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.cm as cm
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pylab import hist, show
import scipy
import zipfile
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
import seaborn as sn
from sklearn.metrics import silhouette_samples, silhouette_score


sys.setrecursionlimit(100000)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)

### import 2011-2012 nhl data set

In [2]:
da = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/data/nhl_pbp20112012.csv')
#da = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/data/nhl_pbp20112012.csv')
da = da.drop('Unnamed: 0', axis=1)

- rename columns and create a season variable

In [3]:
da = da.rename(columns={ 'Game_Id' : 'GameNumber', 'Ev_Zone': 'EventZone', 'Time_Elapsed': 'Time', 'Ev_Team': 'EventTeamCode', 'Description': 'EventDetail', 'Away_Team': 'ATeamCode', 'Home_Team':'HTeamCode', 'p1_name': 'EventPlayer1','p2_name': 'EventPlayer2', 'p3_name': 'EventPlayer3', 'awayPlayer1': 'APlayer1', 'awayPlayer2': 'APlayer2', 'awayPlayer3': 'APlayer3', 'awayPlayer4': 'APlayer4', 'awayPlayer5': 'APlayer5', 'awayPlayer6': 'APlayer6', 'homePlayer1': 'HPlayer1', 'homePlayer2': 'HPlayer2', 'homePlayer3': 'HPlayer3','homePlayer4': 'HPlayer4', 'homePlayer5': 'HPlayer5','homePlayer6': 'HPlayer6', 'Away_Score': 'AGoals', 'Home_Score': 'HGoals', 'Away_Goalie': 'AGoalie', 'Home_Goalie': 'HGoalie', 'Away_Players': 'PlayersA', 'Home_Players': 'PlayersH' })
da['Season'] = 2011
da = da[['Season', 'GameNumber', 'Date', 'Period', 'Time', 'Strength', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'ATeamCode', 'APlayer1', 'APlayer2', 'APlayer3', 'APlayer4', 'APlayer5', 'APlayer6', 'AGoalie', 'PlayersA', 'HTeamCode', 'HPlayer1', 'HPlayer2', 'HPlayer3', 'HPlayer4', 'HPlayer5', 'HPlayer6', 'HGoalie', 'PlayersH']]

- keep only regular season games and exclude irrelevant on-ice events

In [4]:
da = da[da['GameNumber'] <= 21230]
da = da[da['Period'] <= 4]
da = da[da['Period'] >= 1]
da = da[da['Event']!='STOP']
da = da[da['Event']!='EISTR']
da = da[da['Event']!='EIEND']
da = da[da['Event']!='PSTR']
da = da[da['Event']!='PEND']
da = da[da['Event']!='SOC']
da = da[da['Event']!='GEND']

- create an event number variable that will count the number of events per game. 
- create an advantage type variable for even strength, power play and short handed situations.

In [5]:
da['EventNumber'] = da.groupby(['Season', 'GameNumber']).cumcount()+1

In [6]:
da['AdvantageType'] = da.apply(lambda x: 'EV' if ((x['PlayersA'] == x['PlayersH']) & (x['PlayersA'] != 1) & (x['PlayersH'] != 1)) else 'PP' if ((x['PlayersA'] > x['PlayersH']) & (x['EventTeamCode'] == x['ATeamCode'])) else 'PP' if ((x['PlayersA'] < x['PlayersH']) & (x['EventTeamCode'] == x['HTeamCode'])) else 'SH' if ((x['PlayersA'] < x['PlayersH']) & (x['EventTeamCode'] == x['ATeamCode'])) else 'SH' if ((x['PlayersA'] > x['PlayersH']) & (x['EventTeamCode'] == x['HTeamCode'])) else 'PP' if ((x['PlayersA'] == 1 ) & (x['PlayersH'] == 1)) else np.nan, axis=1) 

In [7]:
da['AdvantageType'].value_counts()

EV    258886
PP     34787
SH     16425
Name: AdvantageType, dtype: int64

- reshape data from wide to long for away team players and home team players

In [8]:
db = da.copy()
a = [col for col in db.columns if 'APlayer' in col]
b = [col for col in db.columns if 'HPlayer' in col]
db = pd.lreshape(db, {'APlayer' : a, 'HPlayer' : b })
db.head()

,AGoalie,ATeamCode,AdvantageType,Date,Event,EventDetail,EventNumber,EventPlayer1,EventPlayer2,EventPlayer3,EventTeamCode,GameNumber,HGoalie,HTeamCode,Period,PlayersA,PlayersH,Season,Strength,Time,APlayer,HPlayer
0,ILYA BRYZGALOV,PHI,EV,2011-10-06,FAC,BOS won Neu. Zone - PHI #48 BRIERE vs BOS #46 ...,1,DAVID KREJCI,DANNY BRIERE,NaN,BOS,20001,TIM THOMAS,BOS,1,6,6,2011,5x5,0:00,DANNY BRIERE,DAVID KREJCI
1,ILYA BRYZGALOV,PHI,EV,2011-10-06,GIVE,"PHI GIVEAWAY - #93 VORACEK, Neu. Zone",2,JAKUB VORACEK,NaN,NaN,PHI,20001,TIM THOMAS,BOS,1,6,6,2011,5x5,0:14,DANNY BRIERE,DAVID KREJCI
2,ILYA BRYZGALOV,PHI,EV,2011-10-06,SHOT,"BOS ONGOAL - #44 SEIDENBERG, Snap, Off. Zone, ...",3,DENNIS SEIDENBERG,NaN,NaN,BOS,20001,TIM THOMAS,BOS,1,6,6,2011,5x5,0:47,DANNY BRIERE,DAVID KREJCI
3,ILYA BRYZGALOV,PHI,EV,2011-10-06,MISS,"BOS #18 HORTON, Wrist, Wide of Net, Off. Zone,...",4,NATHAN HORTON,NaN,NaN,BOS,20001,TIM THOMAS,BOS,1,6,6,2011,5x5,1:01,DANNY BRIERE,DAVID KREJCI
4,ILYA BRYZGALOV,PHI,EV,2011-10-06,HIT,"PHI #17 SIMMONDS HIT BOS #55 BOYCHUK, Neu. Zone",5,WAYNE SIMMONDS,JOHNNY BOYCHUK,NaN,PHI,20001,TIM THOMAS,BOS,1,6,6,2011,5x5,1:12,DANNY BRIERE,PATRICE BERGERON


In [9]:
db = db[['Season', 'GameNumber', 'Date', 'Period', 'Time', 'AdvantageType', 'Strength', 'EventNumber', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'ATeamCode', 'APlayer', 'AGoalie', 'PlayersA', 'HTeamCode', 'HPlayer', 'HGoalie', 'PlayersH']]
db = db.sort_values(['Season', 'GameNumber', 'Period', 'Time'], ascending=[True, True, True, True])

- reshape data from wide to long for away and home team code and player respectfully.

In [10]:
dc = db.copy()
dc = dc.rename(columns={'EventTeamCode': 'EventTeam', 'EventPlayer1': 'EventP1', 'EventPlayer2': 'EventP2', 'EventPlayer3': 'EventP3', 'PlayersA':'PA', 'PlayersH': 'PH'})

In [11]:
a = [col for col in dc.columns if 'Code' in col]
b = [col for col in dc.columns if 'Player' in col]
c = [col for col in dc.columns if 'Goalie' in col]
d = [col for col in dc.columns if 'Position' in col]
dc = pd.lreshape(dc, {'TeamCode' : a, 'PlayerName' : b, 'Goalie': c})

In [12]:
dc = dc.rename(columns={'EventTeam': 'EventTeamCode', 'EventP1': 'EventPlayer1', 'EventP2': 'EventPlayer2', 'EventP3': 'EventPlayer3', 'PA':'PlayersA', 'PH': 'PlayersH'})

In [13]:
dc.shape

(3561344, 19)

### games played

- calculate the games each player participated in for the duration of the 2011 regular season.

In [14]:
s = dc.copy()
s = s[['Season', 'GameNumber', 'TeamCode', 'PlayerName']]
s = s.sort_values(['Season', 'GameNumber', 'TeamCode'], ascending=[True, True, True])
s = s.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])

In [15]:
s['GP'] = s.groupby(['Season', 'TeamCode','PlayerName'])['GameNumber'].transform('count')
s = s.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
s = s[['Season', 'TeamCode', 'PlayerName', 'GP']]
s.head()

,Season,TeamCode,PlayerName,GP
1779940,2011,BOS,DAVID KREJCI,79
1779941,2011,BOS,NATHAN HORTON,46
1779942,2011,BOS,MILAN LUCIC,81
1779943,2011,BOS,ANDREW FERENCE,72
1779944,2011,BOS,DENNIS SEIDENBERG,80


In [16]:
s.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
GP            0
dtype: int64

In [17]:
s.shape

(1056, 4)

### shots for and against

- calculate the shots against each player faced througout the duration of a season. 
- create shot for and shot against variables. If event team code is indiferrent to team code assign a value of 1. If not, assign a value of 0. 
- **goals are a result of a shot and therefore need to be added to a player's shot total**. Display total shots per player for a season.

In [18]:
sa = dc.copy()
sa = sa[sa['Event'].isin(['SHOT','GOAL'])]

In [19]:
sa['ShotF'] = sa.apply(lambda x: 1 if x['EventTeamCode'] == x['TeamCode'] else 0, axis=1)
sa['ShotsF'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['ShotF'].transform('sum')

sa['ShotA'] = sa.apply(lambda x: 1 if x['EventTeamCode'] != x['TeamCode'] else 0, axis=1)
sa['ShotsA'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['ShotA'].transform('sum')

- create shot for and shot against variables for even strength situations only. If event team code is indiferrent to team code assign a value of 1. If not, assign a value of 0. 
- **goals are a result of a shot and therefore need to be added to a player's shot total**. Display total shots per player for a season.

In [20]:
sa['EVShotF'] = sa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] == x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] == x['TeamCode'])) else 0, axis=1)
sa['EVShotsF'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVShotF'].transform('sum')

sa['EVShotA'] = sa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] != x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] != x['TeamCode'])) else 0, axis=1)
sa['EVShotsA'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVShotA'].transform('sum')

- calculate the shot differential for all shots and even strength shots.

In [21]:
sa['DShots'] = sa['ShotsF'] - sa['ShotsA']
sa['EVDShots'] = sa['EVShotsF'] - sa['EVShotsA']

In [22]:
sa = sa [['Season', 'TeamCode', 'PlayerName', 'ShotsF', 'ShotsA', 'DShots', 'EVShotsF', 'EVShotsA', 'EVDShots' ]]
sa = sa.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
sa.head()

,Season,TeamCode,PlayerName,ShotsF,ShotsA,DShots,EVShotsF,EVShotsA,EVDShots
12,2011,PHI,DANNY BRIERE,678,512,166,499,511,-12
13,2011,PHI,WAYNE SIMMONDS,786,550,236,519,550,-31
14,2011,PHI,JAKUB VORACEK,774,503,271,531,475,56
15,2011,PHI,CHRIS PRONGER,158,125,33,109,83,26
16,2011,PHI,MATT CARLE,976,807,169,819,743,76


In [23]:
sa.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
ShotsF        0
ShotsA        0
DShots        0
EVShotsF      0
EVShotsA      0
EVDShots      0
dtype: int64

In [24]:
sa.shape

(1055, 9)

### goals for and against

- create goals for and goals against variable. If event team code is the same as team code assign a value of 1 for goals for and a value of 0 for goals against. If event team code is different to team code, assign a value of 0 for golas for and a value of 1 for goals against. Display total goals for and against per player for a season.

In [25]:
gfa = dc.copy()
gfa = gfa[gfa['Event'] == 'GOAL']

In [26]:
gfa['EVGoalF'] = gfa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] == x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] == x['TeamCode'])) else 0, axis=1)
gfa['EVGoalA'] = gfa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] != x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] != x['TeamCode'])) else 0, axis=1)

In [27]:
gfa['GoalF'] = gfa.apply(lambda x: 1 if (x['EventTeamCode'] == x['TeamCode']) else 0, axis=1)
gfa['GoalA'] = gfa.apply(lambda x: 1 if (x['EventTeamCode'] != x['TeamCode']) else 0, axis=1)


In [28]:
gfa['EVGoalsF'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVGoalF'].transform('sum')
gfa['EVGoalsA'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVGoalA'].transform('sum')
gfa['GoalsF'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['GoalF'].transform('sum')
gfa['GoalsA'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['GoalA'].transform('sum')

In [29]:
gfa = gfa[['Season', 'GameNumber', 'TeamCode', 'PlayerName' , 'GoalsF', 'GoalsA', 'EVGoalsF', 'EVGoalsA']]
gfa = gfa.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
gfa['Plus/Minus'] = gfa['EVGoalsF'] - gfa['EVGoalsA']
gfa = gfa.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
gfa = gfa[['Season', 'TeamCode', 'PlayerName' , 'GoalsF', 'GoalsA', 'EVGoalsF', 'EVGoalsA', 'Plus/Minus']]
gfa.head()

,Season,TeamCode,PlayerName,GoalsF,GoalsA,EVGoalsF,EVGoalsA,Plus/Minus
226,2011,PHI,WAYNE SIMMONDS,76,43,43,43,0
228,2011,PHI,CLAUDE GIROUX,108,74,66,55,11
230,2011,PHI,JAROMIR JAGR,81,43,49,43,6
232,2011,PHI,CHRIS PRONGER,18,12,9,7,2
234,2011,PHI,ANDREJ MESZAROS,64,62,55,46,9


In [30]:
gfa.shape

(1021, 8)

### create dataframes for all on-ice event types

#### goals dataframe (dg)

In [31]:
dg = dc.copy()
dg = dg[dg['Event'] == 'GOAL']

In [32]:
dg = dg[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dg = dg.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a goal variable. If event type is a goal assign a value of 1. If not, assign a value of 0. Display total goals per player for a season.

In [33]:
dg['Goal'] = dg.apply(lambda x: 1 if (x['Event'] == 'GOAL') else 0, axis=1)
dg = dg.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dg['Goals'] = dg.groupby(['Season','TeamCode', 'PlayerName'])['Goal'].transform('sum')

- keep one observation per player by game and drop duplicates. This purpose of this step is to calculate the quantity of games per player for a season. 

In [34]:
dg = dg.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dg = dg.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dg = dg[['Season', 'TeamCode', 'PlayerName', 'Goals']]
dg.head()

,Season,TeamCode,PlayerName,Goals
226,2011,PHI,CLAUDE GIROUX,28
243,2011,PHI,JAKUB VORACEK,18
457,2011,BOS,BRAD MARCHAND,28
1732,2011,TOR,MATTHEW LOMBARDI,8
2468,2011,TOR,DION PHANEUF,12


In [35]:
dg.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Goals         0
dtype: int64

In [36]:
dg.shape

(734, 4)

#### assists dataframe (dast)

In [37]:
dast = dc.copy()
dast = dast[dast['Event'] == 'GOAL']

In [38]:
dast = dast[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer2', 'EventPlayer3']]
dast = dast.rename(columns={'EventPlayer2': 'Assist1Name', 'EventPlayer3': 'Assist2Name', 'EventTeamCode':'TeamCode'})

- reshape assist players 1 and 2 to create one column of assist players

In [39]:
a = [col for col in dast.columns if 'Name' in col]
dast = pd.lreshape(dast, {'PlayerName' : a})

- create an assist variable. If event type is a goal assign a value of 1. If not, assign a value of 0. Display total goals per player for a season.

In [40]:
dast['Assist'] = dast.apply(lambda x: 1 if (x['Event'] == 'GOAL') else 0, axis=1)
dast = dast.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dast['Assists'] = dast.groupby(['Season','TeamCode', 'PlayerName'])['Assist'].transform('sum')

- keep one observation per player by game and drop duplicates. This purpose of this step is to calculate the quantity of games per player for a season. 

In [41]:
dast = dast.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dast = dast.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dast = dast[['Season', 'TeamCode', 'PlayerName', 'Assists']]
dast.head()

,Season,TeamCode,PlayerName,Assists
0,2011,PHI,JAROMIR JAGR,35
5,2011,PHI,ANDREJ MESZAROS,18
11,2011,BOS,TYLER SEGUIN,38
16,2011,TOR,MIKE BROWN,2
21,2011,TOR,PHIL KESSEL,45


In [42]:
dast.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Assists       0
dtype: int64

In [43]:
dast.shape

(840, 4)

#### shot dataframe (ds)

In [44]:
ds = dc.copy()
ds = ds[ds['Event'] == 'SHOT']

In [45]:
ds = ds[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
ds = ds.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})
ds.head()

,Season,GameNumber,EventNumber,Event,TeamCode,PlayerName
12,2011,20001,3,SHOT,BOS,DENNIS SEIDENBERG
13,2011,20001,3,SHOT,BOS,DENNIS SEIDENBERG
14,2011,20001,3,SHOT,BOS,DENNIS SEIDENBERG
15,2011,20001,3,SHOT,BOS,DENNIS SEIDENBERG
16,2011,20001,3,SHOT,BOS,DENNIS SEIDENBERG


- create a shot variable. If event type is a goal assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [46]:
ds['Shot'] = ds.apply(lambda x: 1 if (x['Event'] == 'SHOT') else 0, axis=1)
ds = ds.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
ds['Shots'] = ds.groupby(['Season','TeamCode', 'PlayerName'])['Shot'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [47]:
ds = ds.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
ds = ds.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
ds = ds[['Season', 'TeamCode', 'PlayerName', 'Shots']]
ds.head()

,Season,TeamCode,PlayerName,Shots
12,2011,BOS,DENNIS SEIDENBERG,169
18,2011,PHI,CLAUDE GIROUX,214
72,2011,BOS,TYLER SEGUIN,213
96,2011,BOS,SHAWN THORNTON,109
114,2011,PHI,JAMES VAN RIEMSDYK,110


In [48]:
ds.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Shots         0
dtype: int64

In [49]:
ds.shape

(938, 4)

#### block dataframe (db)

In [50]:
dbl = dc.copy()
dbl = dbl[dbl['Event'] == 'BLOCK']

In [51]:
dbl['BlockTeam'] = dbl.EventDetail.str.split(' ', expand = True)[6]
dbl['BlockTeamCode'] = dbl.BlockTeam.str.split(' ', expand = True)[0]

In [52]:
dbl = dbl[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventPlayer1', 'BlockTeamCode']]
dbl = dbl.rename(columns={'EventPlayer1': 'PlayerName', 'BlockTeamCode':'TeamCode'})

- create a block variable. If event type is a block assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [53]:
dbl['Block'] = dbl.apply(lambda x: 1 if (x['Event'] == 'BLOCK') else 0, axis=1)
dbl = dbl.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dbl['Blocks'] = dbl.groupby(['Season','TeamCode', 'PlayerName'])['Block'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [54]:
dbl = dbl.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dbl = dbl.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dbl = dbl[['Season', 'TeamCode', 'PlayerName', 'Blocks']]
dbl.head()

,Season,TeamCode,PlayerName,Blocks
60,2011,BOS,MATT BARTKOWSKI,2
126,2011,BOS,ANDREW FERENCE,81
156,2011,PHI,WAYNE SIMMONDS,33
162,2011,PHI,KIMMO TIMONEN,127
186,2011,PHI,CLAUDE GIROUX,42


In [55]:
dbl.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Blocks        0
dtype: int64

In [56]:
dbl.shape

(1066, 4)

#### hit dataframe (dh)

In [57]:
dhit = dc.copy()
dhit = dhit[dhit['Event'] == 'HIT']

In [58]:
dhit = dhit[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dhit = dhit.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a hit variable. If event type is a hit assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [59]:
dhit['Hit'] = dhit.apply(lambda x: 1 if (x['Event'] == 'HIT') else 0, axis=1)
dhit = dhit.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dhit['Hits'] = dhit.groupby(['Season','TeamCode', 'PlayerName'])['Hit'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [60]:
dhit = dhit.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dhit = dhit.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dhit = dhit[['Season', 'TeamCode', 'PlayerName', 'Hits']]
dhit.head()

,Season,TeamCode,PlayerName,Hits
54,2011,BOS,ANDREW FERENCE,66
78,2011,BOS,DENNIS SEIDENBERG,154
84,2011,BOS,SHAWN THORNTON,91
144,2011,PHI,MAX TALBOT,150
261,2011,PHI,WAYNE SIMMONDS,140


In [61]:
dhit.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Hits          0
dtype: int64

In [62]:
dhit.shape

(972, 4)

#### penalty dataframe (dpen)

In [63]:
dp = dc.copy()
dp = dp[dp['Event'] == 'PENL']

- use event detail to find the duration of a given penalty and to assign it to the proper player. Major penalty is 5 minutes, so a value of 5 is assigned for every event that had a major penalty.

In [64]:
dp['Pen'] = dp.EventDetail.str.split(')', expand = True)[0]
dp['Penal'] = dp.Pen.str.split('(', expand = True)[1]
dp['Penalt'] = dp.Penal.str.split(' ', expand = True)[0]
dp['Penalty'] = dp.apply(lambda x: 5 if x['Penalt'] == 'maj' else x['Penalt'], axis=1)
dp['Penalty'] = dp['Penalty'].convert_objects(convert_numeric=True)
dp = dp[['Season', 'GameNumber', 'Period', 'Time', 'EventNumber', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'TeamCode', 'PlayerName', 'Penalty']]

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


- use event detail to find penalty team code and penalty player name.

In [65]:
dp['PenaltyTeamCode'] = dp.EventDetail.str.split('\\s', expand = True)[0]
dp['PN'] = dp.EventDetail.str.split('\\s', expand = True)[1]
dp['PNumber'] = dp.PN.str.split('#', expand = True)[1]
dp['PenaltyPlayerNumber'] = dp.PNumber.str.split(' ', expand = True)[0]
dp['PenaltyPlayerLName'] = dp.EventDetail.str.split('\\s', expand = True)[2]
#dp['PenaltyPlayerLName'] = dp.PPlLN.str.split(' ', expand = True)[0]
dp = dp[['Season', 'GameNumber', 'Period', 'Time', 'EventNumber', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'TeamCode', 'PlayerName', 'Penalty', 'PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyPlayerLName']]

- seperate player name into first and last. The purpose is to connect the correct player name to penalty player name.

In [66]:
dp['PlayerFName'] = dp.PlayerName.str.split('\\s', expand = True)[0]
dp['PlayerLName'] = dp.PlayerName.str.split('\\s', expand = True)[1]
dp['PenaltyPlayerFName'] = dp.apply(lambda x: x['PlayerFName'] if ((x['PenaltyTeamCode'] == x['TeamCode']) & (x['PenaltyPlayerNumber'] == x['PenaltyPlayerNumber']) & (x['PlayerLName'] == x['PenaltyPlayerLName'])) else np.nan, axis=1)
dp['PenaltyPlayerFName'] = dp.groupby(['Season','PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyPlayerLName'])['PenaltyPlayerFName'].apply(lambda x: x.ffill().bfill())

In [67]:
dp['PenaltyName'] = dp.apply(lambda x: x['PlayerName'] if ((x['PenaltyTeamCode'] == x['TeamCode']) & (x['PenaltyPlayerNumber'] == x['PenaltyPlayerNumber']) & (x['PlayerLName'] == x['PenaltyPlayerLName']) & (x['PlayerFName'] == x['PenaltyPlayerFName'])) else np.nan, axis=1)
dp['PenaltyName'] = dp.groupby(['Season','PenaltyTeamCode', 'PenaltyPlayerLName', 'PenaltyPlayerNumber', 'PenaltyPlayerLName'])['PenaltyName'].apply(lambda x: x.ffill().bfill())

- drop duplicates per game and event number. Calculate the total number of penalties for each player.

In [68]:
dp = dp.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyName'])
dp['Penalties'] = dp.groupby(['Season', 'PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyName'])['Penalty'].transform('sum')
dp['Penalties'] = dp.groupby(['Season','PenaltyTeamCode', 'PenaltyPlayerNumber', 'PlayerName'])['Penalties'].apply(lambda x: x.ffill().bfill())
#dp.head()

- drop duplicates to keep one observation per player for the season.

In [69]:
dp = dp[['Season', 'PenaltyTeamCode', 'PenaltyName', 'Penalties']]
dp = dp.drop_duplicates(['Season', 'PenaltyTeamCode', 'PenaltyName'])
dp = dp.rename(columns={'PenaltyTeamCode':'TeamCode', 'PenaltyName':'PlayerName'})
dp = dp[['Season', 'TeamCode', 'PlayerName', 'Penalties']]
dp.head()

,Season,TeamCode,PlayerName,Penalties
210,2011,BOS,NATHAN HORTON,52.0
441,2011,PHI,ZAC RINALDO,222.0
546,2011,BOS,DENNIS SEIDENBERG,39.0
578,2011,PHI,DANNY BRIERE,69.0
649,2011,PHI,WAYNE SIMMONDS,114.0


- keep observations that are not null. Since a team can be allocated a penalty, those observations are excluded from the data.

In [70]:
dp = dp[dp['PlayerName'].notnull()]

In [71]:
dp.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Penalties     0
dtype: int64

In [72]:
dp.shape

(900, 4)

####  miss dataframe (dmiss)

In [73]:
dmiss = dc.copy()
dmiss = dmiss[dmiss['Event'] == 'MISS']

In [74]:
dmiss = dmiss[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dmiss = dmiss.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a miss variable. If event type is a miss assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [75]:
dmiss['Miss'] = dmiss.apply(lambda x: 1 if (x['Event'] == 'MISS') else 0, axis=1)
dmiss = dmiss.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dmiss['Misses'] = dmiss.groupby(['Season','TeamCode', 'PlayerName'])['Miss'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [76]:
dmiss = dmiss.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dmiss = dmiss.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dmiss = dmiss[['Season', 'TeamCode', 'PlayerName', 'Misses']]
dmiss.head()

,Season,TeamCode,PlayerName,Misses
30,2011,PHI,JAKUB VORACEK,62
42,2011,BOS,TYLER SEGUIN,89
90,2011,BOS,SHAWN THORNTON,38
120,2011,PHI,JAROMIR JAGR,80
192,2011,BOS,JOHNNY BOYCHUK,72


In [77]:
dmiss.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Misses        0
dtype: int64

In [78]:
dmiss.shape

(898, 4)

####  takeaway dataframe (dtake)

In [79]:
dtake = dc.copy()
dtake = dtake[dtake['Event'] == 'TAKE']

In [80]:
dtake = dtake[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dtake = dtake.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a takeaway variable. If event type is a takeaway assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [81]:
dtake['Takeaway'] = dtake.apply(lambda x: 1 if (x['Event'] == 'TAKE') else 0, axis=1)
dtake = dtake.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dtake['Takeaways'] = dtake.groupby(['Season','TeamCode', 'PlayerName'])['Takeaway'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [82]:
dtake = dtake.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dtake = dtake.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dtake = dtake[['Season', 'TeamCode', 'PlayerName', 'Takeaways']]
dtake.head()

,Season,TeamCode,PlayerName,Takeaways
150,2011,BOS,JORDAN CARON,6
339,2011,BOS,BRAD MARCHAND,40
357,2011,BOS,DENNIS SEIDENBERG,17
405,2011,BOS,PATRICE BERGERON,55
604,2011,PHI,MAX TALBOT,30


In [83]:
dtake.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Takeaways     0
dtype: int64

In [84]:
dtake.shape

(868, 4)

####  giveaway dataframe (dgive)

In [85]:
dgive = dc.copy()
dgive = dgive[dgive['Event'] == 'GIVE']

In [86]:
dgive = dgive[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dgive = dgive.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a giveaway variable. If event type is a giveaway assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [87]:
dgive['Giveaway'] = dgive.apply(lambda x: 1 if (x['Event'] == 'GIVE') else 0, axis=1)
dgive = dgive.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dgive['Giveaways'] = dgive.groupby(['Season','TeamCode', 'PlayerName'])['Giveaway'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [88]:
dgive = dgive.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dgive = dgive.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dgive = dgive[['Season', 'TeamCode', 'PlayerName', 'Giveaways']]
dgive.head()

,Season,TeamCode,PlayerName,Giveaways
6,2011,PHI,JAKUB VORACEK,38
66,2011,BOS,TYLER SEGUIN,42
108,2011,PHI,SCOTT HARTNELL,42
138,2011,BOS,PATRICE BERGERON,25
180,2011,PHI,MATT CARLE,55


In [89]:
dgive.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Giveaways     0
dtype: int64

In [90]:
dgive.shape

(949, 4)

### import 2011 player shifts

In [91]:
dshift = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/data/nhl_shifts20112012.csv')
#dshift = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/data/nhl_shifts20112012.csv')
dshift = dshift.drop('Unnamed: 0', axis=1)

- rename variables, keep only regular season games and create a season variable.

In [92]:
dshift= dshift.rename(columns={ 'Game_Id' : 'GameNumber', 'Team': 'TeamCode', 'Player': 'PlayerName' })
dshift = dshift[dshift['GameNumber'] <= 21230]
dshift['Season'] = 2011
dshift = dshift[['Season', 'Date', 'GameNumber', 'Period', 'TeamCode', 'PlayerName', 'Player_Id', 'Start', 'End', 'Duration']]
dshift.head()

,Season,Date,GameNumber,Period,TeamCode,PlayerName,Player_Id,Start,End,Duration
0,2011,2011-10-06,20001,1,BOS,ANDREW FERENCE,8466333,0.0,34.0,34.0
1,2011,2011-10-06,20001,1,BOS,DENNIS SEIDENBERG,8469619,0.0,52.0,52.0
2,2011,2011-10-06,20001,1,BOS,MILAN LUCIC,8473473,0.0,56.0,56.0
3,2011,2011-10-06,20001,1,BOS,DAVID KREJCI,8471276,0.0,69.0,69.0
4,2011,2011-10-06,20001,1,BOS,NATHAN HORTON,8470596,0.0,69.0,69.0


- calcuate the seconds an individual played per game. Convert seconds to minutes and drop duplicates per season, game number, team and player.

In [93]:
dshift['Seconds'] = dshift.groupby(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])['Duration'].transform('sum')
dshift['Minutes'] = dshift['Seconds']/60
dshift['Minutes'] = dshift['Minutes'].round(2)

In [94]:
dshift = dshift.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])

In [95]:
dshift.head()

,Season,Date,GameNumber,Period,TeamCode,PlayerName,Player_Id,Start,End,Duration,Seconds,Minutes
0,2011,2011-10-06,20001,1,BOS,ANDREW FERENCE,8466333,0.0,34.0,34.0,1059.0,17.65
1,2011,2011-10-06,20001,1,BOS,DENNIS SEIDENBERG,8469619,0.0,52.0,52.0,1384.0,23.07
2,2011,2011-10-06,20001,1,BOS,MILAN LUCIC,8473473,0.0,56.0,56.0,846.0,14.10
3,2011,2011-10-06,20001,1,BOS,DAVID KREJCI,8471276,0.0,69.0,69.0,1050.0,17.50
4,2011,2011-10-06,20001,1,BOS,NATHAN HORTON,8470596,0.0,69.0,69.0,985.0,16.42


- calcuate the seconds an individual played for the whole regular season. Convert seconds to minutes and drop duplicates per season, game number, team and player.

In [96]:
dshift['TSeconds'] = dshift.groupby(['Season', 'TeamCode', 'PlayerName'])['Seconds'].transform('sum')
dshift['TOI'] = dshift['TSeconds']/60
dshift['TOI'] = dshift['TOI'].round(2)
dshift.head()

,Season,Date,GameNumber,Period,TeamCode,PlayerName,Player_Id,Start,End,Duration,Seconds,Minutes,TSeconds,TOI
0,2011,2011-10-06,20001,1,BOS,ANDREW FERENCE,8466333,0.0,34.0,34.0,1059.0,17.65,81597.0,1359.95
1,2011,2011-10-06,20001,1,BOS,DENNIS SEIDENBERG,8469619,0.0,52.0,52.0,1384.0,23.07,115388.0,1923.13
2,2011,2011-10-06,20001,1,BOS,MILAN LUCIC,8473473,0.0,56.0,56.0,846.0,14.10,82761.0,1379.35
3,2011,2011-10-06,20001,1,BOS,DAVID KREJCI,8471276,0.0,69.0,69.0,1050.0,17.50,87326.0,1455.43
4,2011,2011-10-06,20001,1,BOS,NATHAN HORTON,8470596,0.0,69.0,69.0,985.0,16.42,43975.0,732.92


- keep only observation per player for the season

In [97]:
dshift = dshift.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dshift = dshift[['Season', 'TeamCode', 'PlayerName', 'TOI']]
dshift.head()

,Season,TeamCode,PlayerName,TOI
0,2011,BOS,ANDREW FERENCE,1359.95
1,2011,BOS,DENNIS SEIDENBERG,1923.13
2,2011,BOS,MILAN LUCIC,1379.35
3,2011,BOS,DAVID KREJCI,1455.43
4,2011,BOS,NATHAN HORTON,732.92


In [98]:
dshift.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
TOI           0
dtype: int64

In [99]:
dshift.shape

(1056, 4)

### import 2011 player positions

In [100]:
dpos = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/data/nhl_positions20112012.csv')
#dpos = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/data/nhl_positions20112012.csv')

In [101]:
dpos= dpos.rename(columns={ 'Team': 'TeamCode', 'Old Team' : 'OldTeam', 'Pos ': 'Position'})
dpos.head()

,PlayerNumber,Last Name,First Name,TeamCode,OldTeam,OGP,Position,PlayerFullName,PlayerName
0,8.0,Abdelkader,Justin,DET,NaN,NaN,LW/C,Justin Abdelkader,JUSTIN ABDELKADER
1,72.0,Adam,Luke,BUF,NaN,NaN,C/LW,Luke Adam,LUKE ADAM
2,27.0,Adams,Craig,PIT,NaN,NaN,RW,Craig Adams,CRAIG ADAMS
3,41.0,Alberts,Andrew,VAN,NaN,NaN,D,Andrew Alberts,ANDREW ALBERTS
4,11.0,Alfredsson,Daniel,OTT,NaN,NaN,RW,Daniel Alfredsson,DANIEL ALFREDSSON


In [102]:
dpos= dpos.rename(columns={ 'Team': 'TeamCode', 'Pos ': 'Position'})
dpos['Season'] = 2011
dpos = dpos[['Season', 'TeamCode', 'PlayerName', 'Position']]
dpos.head()

,Season,TeamCode,PlayerName,Position
0,2011,DET,JUSTIN ABDELKADER,LW/C
1,2011,BUF,LUKE ADAM,C/LW
2,2011,PIT,CRAIG ADAMS,RW
3,2011,VAN,ANDREW ALBERTS,D
4,2011,OTT,DANIEL ALFREDSSON,RW


In [103]:
dpos.dtypes

Season         int64
TeamCode      object
PlayerName    object
Position      object
dtype: object

In [104]:
dpos['Position'] = dpos.apply(lambda x: 'C' if x['Position'] == 'C/LW' else 'C' if x['Position'] == 'C/RW' else 'C' if x['Position'] == 'C/RW/LW' else 'C' if x['Position'] == 'C/LW/RW' else 'W' if x['Position'] == 'RW' else 'W' if x['Position'] == 'LW' else 'W' if x['Position'] == 'LW/C' else 'W' if x['Position'] == 'RW/C' else 'W' if x['Position'] == 'RW/LW' else 'W' if x['Position'] == 'LW/RW' else 'W' if x['Position'] == 'RW/LW/C' else 'W' if x['Position'] == 'RW/C/LW' else 'W' if x['Position'] == 'RW/C/LW' else 'W' if x['Position'] == 'LW/D' else 'W' if x['Position'] == 'LW/C/RW' else 'W' if x['Position'] == 'LW/RW/C' else 'D' if x['Position'] == 'D/LW' else x['Position'] , axis=1)
dpos['Position'].value_counts()

W    350
D    297
C    247
G     89
Name: Position, dtype: int64

In [105]:
dpos['TeamCode'] = dpos.apply(lambda x: 'L.A' if x['TeamCode'] == 'LAK' else 'N.J' if x['TeamCode'] == 'NJD' else 'S.J' if x['TeamCode'] == 'SJS' else 'T.B' if x['TeamCode'] == 'TBL' else x['TeamCode'] , axis=1)
dpos.head()

,Season,TeamCode,PlayerName,Position
0,2011,DET,JUSTIN ABDELKADER,W
1,2011,BUF,LUKE ADAM,C
2,2011,PIT,CRAIG ADAMS,W
3,2011,VAN,ANDREW ALBERTS,D
4,2011,OTT,DANIEL ALFREDSSON,W


In [106]:
dpos.shape

(983, 4)

### merge data frames

- merge games played, goals and assists.

In [107]:
dd = pd.merge(s, dpos, on=['Season', 'TeamCode', 'PlayerName'], how='left').merge(dg,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dast,on=['Season', 'TeamCode', 'PlayerName'], how = 'left')


- forward and backward fill players that have been traded during the season. Fill in positions for three players.

In [108]:
dd['Position'] = dd.groupby(['PlayerName'])['Position'].apply(lambda x: x.ffill().bfill())
dd['Position'] = dd.apply(lambda x: 'D' if x['PlayerName'] == 'KEVIN MARSHALL' else 'W' if x['PlayerName'] == 'EVGENY DADONOV' else 'W' if x['PlayerName'] == 'MAXIM MAYOROV' else x['Position'] , axis=1)
dd.head()

,Season,TeamCode,PlayerName,GP,Position,Goals,Assists
0,2011,BOS,DAVID KREJCI,79,C,23.0,39.0
1,2011,BOS,NATHAN HORTON,46,W,17.0,15.0
2,2011,BOS,MILAN LUCIC,81,W,26.0,35.0
3,2011,BOS,ANDREW FERENCE,72,D,6.0,18.0
4,2011,BOS,DENNIS SEIDENBERG,80,D,5.0,18.0


In [109]:
dd.isnull().sum()

Season          0
TeamCode        0
PlayerName      0
GP              0
Position        0
Goals         322
Assists       216
dtype: int64

- fill in goals and assists with 0 for players who had no points.

In [110]:
dd['Goals'] = dd['Goals'].fillna(0)
dd['Assists'] = dd['Assists'].fillna(0)
dd['Points'] = dd['Goals'] + dd['Assists']
dd.head()

,Season,TeamCode,PlayerName,GP,Position,Goals,Assists,Points
0,2011,BOS,DAVID KREJCI,79,C,23.0,39.0,62.0
1,2011,BOS,NATHAN HORTON,46,W,17.0,15.0,32.0
2,2011,BOS,MILAN LUCIC,81,W,26.0,35.0,61.0
3,2011,BOS,ANDREW FERENCE,72,D,6.0,18.0,24.0
4,2011,BOS,DENNIS SEIDENBERG,80,D,5.0,18.0,23.0


In [111]:
dd.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
GP            0
Position      0
Goals         0
Assists       0
Points        0
dtype: int64

In [112]:
de = pd.merge(dd, gfa,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(ds,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(sa,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dbl,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dhit,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dp,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dtake,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dgive,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dmiss,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dshift,on=['Season', 'TeamCode', 'PlayerName'], how = 'left')
de.head()

,Season,TeamCode,PlayerName,GP,Position,Goals,Assists,Points,GoalsF,GoalsA,EVGoalsF,EVGoalsA,Plus/Minus,Shots,ShotsF,ShotsA,DShots,EVShotsF,EVShotsA,EVDShots,Blocks,Hits,Penalties,Takeaways,Giveaways,Misses,TOI
0,2011,BOS,DAVID KREJCI,79,C,23.0,39.0,62.0,87.0,70.0,63.0,64.0,-1.0,123.0,806.0,670.0,136.0,649.0,627.0,22.0,46.0,69.0,36.0,43.0,34.0,57.0,1455.43
1,2011,BOS,NATHAN HORTON,46,W,17.0,15.0,32.0,54.0,33.0,36.0,33.0,3.0,72.0,472.0,312.0,160.0,374.0,309.0,65.0,11.0,32.0,52.0,12.0,12.0,24.0,732.92
2,2011,BOS,MILAN LUCIC,81,W,26.0,35.0,61.0,87.0,54.0,65.0,54.0,11.0,122.0,800.0,591.0,209.0,652.0,591.0,61.0,26.0,201.0,133.0,25.0,46.0,56.0,1379.35
3,2011,BOS,ANDREW FERENCE,72,D,6.0,18.0,24.0,60.0,56.0,55.0,46.0,9.0,102.0,612.0,749.0,-137.0,572.0,637.0,-65.0,81.0,66.0,46.0,7.0,38.0,49.0,1359.95
4,2011,BOS,DENNIS SEIDENBERG,80,D,5.0,18.0,23.0,92.0,84.0,74.0,56.0,18.0,169.0,1023.0,957.0,66.0,854.0,761.0,93.0,152.0,154.0,39.0,17.0,25.0,64.0,1923.13


- fill in with 0 for all nan values since not all players have been on the ice for a goal for or a goal against. 

In [113]:
de['GoalsF'] = de['GoalsF'].fillna(0)
de['GoalsA'] = de['GoalsA'].fillna(0)
de['EVGoalsF'] = de['EVGoalsF'].fillna(0)
de['EVGoalsA'] = de['EVGoalsA'].fillna(0)
de['Plus/Minus'] = de['Plus/Minus'].fillna(0)
de['Shots'] = de['Shots'].fillna(0)
de['Hits'] = de['Hits'].fillna(0)
de['Blocks'] = de['Blocks'].fillna(0)
de['Penalties'] = de['Penalties'].fillna(0)
de['Giveaways'] = de['Giveaways'].fillna(0)
de['Takeaways'] = de['Takeaways'].fillna(0)
de['Misses'] = de['Misses'].fillna(0)

In [114]:
de.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
GP            0
Position      0
Goals         0
Assists       0
Points        0
GoalsF        0
GoalsA        0
EVGoalsF      0
EVGoalsA      0
Plus/Minus    0
Shots         0
ShotsF        1
ShotsA        1
DShots        1
EVShotsF      1
EVShotsA      1
EVDShots      1
Blocks        0
Hits          0
Penalties     0
Takeaways     0
Giveaways     0
Misses        0
TOI           0
dtype: int64

### total player stats for the season

- - players that have been traded need to be evaluated for their season performance. Group by sesaon and player name to calcute the sum of each stat. 

In [115]:
de['TGP'] = de.groupby(['Season', 'PlayerName'])['GP'].transform('sum')
de['TGoals'] = de.groupby(['Season', 'PlayerName'])['Goals'].transform('sum')
de['TAssists'] = de.groupby(['Season', 'PlayerName'])['Assists'].transform('sum')
de['TPoints'] = de.groupby(['Season', 'PlayerName'])['Points'].transform('sum')
de['TGoalsF'] = de.groupby(['Season', 'PlayerName'])['GoalsF'].transform('sum')
de['TGoalsA'] = de.groupby(['Season', 'PlayerName'])['GoalsA'].transform('sum')
de['TEVGoalsF'] = de.groupby(['Season', 'PlayerName'])['EVGoalsF'].transform('sum')
de['TEVGoalsA'] = de.groupby(['Season', 'PlayerName'])['EVGoalsA'].transform('sum')
de['TPlus/Minus'] = de.groupby(['Season', 'PlayerName'])['Plus/Minus'].transform('sum')
de['TShots'] = de.groupby(['Season', 'PlayerName'])['Shots'].transform('sum')
de['TShotsF'] = de.groupby(['Season', 'PlayerName'])['ShotsF'].transform('sum')
de['TShotsA'] = de.groupby(['Season', 'PlayerName'])['ShotsA'].transform('sum')
de['TDShots'] = de.groupby(['Season', 'PlayerName'])['DShots'].transform('sum')
de['TEVShotsF'] = de.groupby(['Season', 'PlayerName'])['EVShotsF'].transform('sum')
de['TEVShotsA'] = de.groupby(['Season', 'PlayerName'])['EVShotsA'].transform('sum')
de['TEVDShots'] = de.groupby(['Season', 'PlayerName'])['EVDShots'].transform('sum')
de['TBlocks'] = de.groupby(['Season', 'PlayerName'])['Blocks'].transform('sum')
de['THits'] = de.groupby(['Season', 'PlayerName'])['Hits'].transform('sum')
de['TPenalties'] = de.groupby(['Season', 'PlayerName'])['Penalties'].transform('sum')
de['TTakeaways'] = de.groupby(['Season', 'PlayerName'])['Takeaways'].transform('sum')
de['TGiveaways'] = de.groupby(['Season', 'PlayerName'])['Giveaways'].transform('sum')
de['TMisses'] = de.groupby(['Season', 'PlayerName'])['Misses'].transform('sum')
de['TTOI'] = de.groupby(['Season', 'PlayerName'])['TOI'].transform('sum')

- divide all stats with total time on ice to scale the data for cluster analysis

In [116]:
de['TOIGoals'] = de['TGoals'] / de['TTOI']
de['TOIAssists'] = de['TAssists'] / de['TTOI']
de['TOIPoints'] = de['TPoints'] / de['TTOI']
de['TOIGoalsF'] = de['TGoalsF'] / de['TTOI']
de['TOIGoalsA'] = de['TGoalsA'] / de['TTOI']
de['TOIEVGoalsF'] = de['TEVGoalsF'] / de['TTOI']
de['TOIEVGoalsA'] = de['TEVGoalsA'] / de['TTOI']
de['TOIPlus/Minus'] = de['TPlus/Minus'] / de['TTOI']
de['TOIShots'] = de['TShots'] / de['TTOI']
de['TOIShotsF'] = de['TShotsF'] / de['TTOI']
de['TOIShotsA'] = de['TShotsA'] / de['TTOI']
de['TOIDShots'] = de['TDShots'] / de['TTOI']
de['TOIEVShotsF'] = de['TEVShotsF'] / de['TTOI']
de['TOIEVShotsA'] = de['TEVShotsA'] / de['TTOI']
de['TOIEVDShots'] = de['TEVDShots'] / de['TTOI']
de['TOIBlocks'] = de['TBlocks'] / de['TTOI']
de['TOIHits'] = de['THits'] / de['TTOI']
de['TOIPenalties'] = de['TPenalties'] / de['TTOI']
de['TOITakeaways'] = de['TTakeaways'] / de['TTOI']
de['TOIGiveaways'] = de['TGiveaways'] / de['TTOI']
de['TOIMisses'] = de['TMisses'] / de['TTOI']

- keep season total variables and scaled variables

In [117]:
de = de[['Season', 'TeamCode', 'PlayerName', 'Position', 'TGP', 'TGoals', 'TAssists', 'TPoints', 'TGoalsF', 'TGoalsA', 'TEVGoalsF', 'TEVGoalsA', 'TPlus/Minus', 'TShots', 'TShotsF', 'TShotsA', 'TDShots', 'TEVShotsF', 'TEVShotsA', 'TEVDShots', 'TBlocks', 'THits', 'TPenalties', 'TTakeaways', 'TGiveaways', 'TMisses', 'TTOI', 'TOIGoals', 'TOIAssists', 'TOIPoints', 'TOIGoalsF', 'TOIGoalsA', 'TOIEVGoalsF', 'TOIEVGoalsA', 'TOIPlus/Minus', 'TOIShots', 'TOIShotsF', 'TOIShotsA', 'TOIDShots', 'TOIEVShotsF', 'TOIEVShotsA', 'TOIEVDShots', 'TOIBlocks', 'TOIHits', 'TOIPenalties', 'TOITakeaways', 'TOIGiveaways', 'TOIMisses']]
de.head()

,Season,TeamCode,PlayerName,Position,TGP,TGoals,TAssists,TPoints,TGoalsF,TGoalsA,TEVGoalsF,TEVGoalsA,TPlus/Minus,TShots,TShotsF,TShotsA,TDShots,TEVShotsF,TEVShotsA,TEVDShots,TBlocks,THits,TPenalties,TTakeaways,TGiveaways,TMisses,TTOI,TOIGoals,TOIAssists,TOIPoints,TOIGoalsF,TOIGoalsA,TOIEVGoalsF,TOIEVGoalsA,TOIPlus/Minus,TOIShots,TOIShotsF,TOIShotsA,TOIDShots,TOIEVShotsF,TOIEVShotsA,TOIEVDShots,TOIBlocks,TOIHits,TOIPenalties,TOITakeaways,TOIGiveaways,TOIMisses
0,2011,BOS,DAVID KREJCI,C,79,23.0,39.0,62.0,87.0,70.0,63.0,64.0,-1.0,123.0,806.0,670.0,136.0,649.0,627.0,22.0,46.0,69.0,36.0,43.0,34.0,57.0,1455.43,0.015803,0.026796,0.042599,0.059776,0.048096,0.043286,0.043973,-0.000687,0.084511,0.553788,0.460345,0.093443,0.445916,0.430801,0.015116,0.031606,0.047409,0.024735,0.029545,0.023361,0.039164
1,2011,BOS,NATHAN HORTON,W,46,17.0,15.0,32.0,54.0,33.0,36.0,33.0,3.0,72.0,472.0,312.0,160.0,374.0,309.0,65.0,11.0,32.0,52.0,12.0,12.0,24.0,732.92,0.023195,0.020466,0.043661,0.073678,0.045025,0.049119,0.045025,0.004093,0.098237,0.643999,0.425694,0.218305,0.510288,0.421601,0.088686,0.015008,0.043661,0.070949,0.016373,0.016373,0.032746
2,2011,BOS,MILAN LUCIC,W,81,26.0,35.0,61.0,87.0,54.0,65.0,54.0,11.0,122.0,800.0,591.0,209.0,652.0,591.0,61.0,26.0,201.0,133.0,25.0,46.0,56.0,1379.35,0.018849,0.025374,0.044224,0.063073,0.039149,0.047124,0.039149,0.007975,0.088447,0.579983,0.428463,0.151521,0.472686,0.428463,0.044224,0.018849,0.145721,0.096422,0.018124,0.033349,0.040599
3,2011,BOS,ANDREW FERENCE,D,72,6.0,18.0,24.0,60.0,56.0,55.0,46.0,9.0,102.0,612.0,749.0,-137.0,572.0,637.0,-65.0,81.0,66.0,46.0,7.0,38.0,49.0,1359.95,0.004412,0.013236,0.017648,0.044119,0.041178,0.040443,0.033825,0.006618,0.075003,0.450017,0.550756,-0.100739,0.420604,0.468400,-0.047796,0.059561,0.048531,0.033825,0.005147,0.027942,0.036031
4,2011,BOS,DENNIS SEIDENBERG,D,80,5.0,18.0,23.0,92.0,84.0,74.0,56.0,18.0,169.0,1023.0,957.0,66.0,854.0,761.0,93.0,152.0,154.0,39.0,17.0,25.0,64.0,1923.13,0.002600,0.009360,0.011960,0.047839,0.043679,0.038479,0.029119,0.009360,0.087878,0.531945,0.497626,0.034319,0.444068,0.395709,0.048359,0.079038,0.080078,0.020279,0.008840,0.013000,0.033279


In [118]:
de.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/stats/2011_player_stats.csv', index='False', sep=',')
#de = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/stats/2011_player_stats.csv', index='False', sep=',')